In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
import datetime
import pyarrow
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from linearmodels.panel import PanelOLS

old_data_folder = '/Users/simonneumeyer/Dropbox/DO-FILES/data/'


Bad key "text.kerning_factor" on line 4 in
/Users/simonneumeyer/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Load data

In [2]:
df_master = pd.read_parquet('../../data/output/firm_year_level.pq', engine='pyarrow')

In [3]:
#df_master[[x for x in df_master.columns if 'exit' in x]]

In [4]:
#df_master.columns.tolist()

## Panel regression

In [5]:
indep_vars = ['Serbian-majority municipality', 'Exporting firm', 'age_bis_bis', 'Number of employees', 'Market concentration']
panel_vars = ['isic_4d', 'period']

df_master_not_na = df_master[indep_vars + panel_vars + ['participation']].dropna()

df_master_not_na = df_master_not_na.set_index(panel_vars).sort_index()

X = sm.add_constant(df_master_not_na[indep_vars], prepend=False)
y = df_master_not_na.participation

mod = PanelOLS(y, X, 
               entity_effects=True, 
               time_effects=True,
               drop_absorbed=True
              )

res = mod.fit(cov_type='robust', 
              cluster_time=True
             )
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:          participation   R-squared:                        0.0150
Estimator:                   PanelOLS   R-squared (Between):              0.0546
No. Observations:              176267   R-squared (Within):               0.0226
Date:                Thu, May 05 2022   R-squared (Overall):              0.0252
Time:                        14:17:35   Log-likelihood                  -9.3e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      534.09
Entities:                         315   P-value                           0.0000
Avg Obs:                       559.58   Distribution:                F(5,175939)
Min Obs:                       1.0000                                           
Max Obs:                    1.295e+04   F-statistic (robust):             415.20
                            

In [6]:
df_master_not_na.to_stata('../../data/output/firm_year_level_no_nans.dta')

/Users/simonneumeyer/opt/anaconda3/lib/python3.7/site-packages/pandas/io/stata.py:2252: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    b'Serbian-majority municipality'   ->   Serbian_majority_municipality
    b'Exporting firm'   ->   Exporting_firm
    b'Number of employees'   ->   Number_of_employees
    b'Market concentration'   ->   Market_concentration

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


## With productivity

In [8]:
indep_vars = ['Productivity', 'Productivity_t_1', 'Serbian-majority municipality', 'Exporting firm', 'age_bis_bis', 'Number of employees', 'Market concentration']
panel_vars = ['isic_4d', 'period']

df_master_not_na = df_master[indep_vars + panel_vars + ['participation']].dropna()

df_master_not_na = df_master_not_na.set_index(panel_vars).sort_index()

X = sm.add_constant(df_master_not_na[indep_vars], prepend=False)
y = df_master_not_na.participation

mod = PanelOLS(y, X, 
               entity_effects=True, 
               time_effects=True, 
               drop_absorbed=True
              )

res = mod.fit(cov_type='robust', 
              cluster_time=True
             )
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:          participation   R-squared:                        0.0130
Estimator:                   PanelOLS   R-squared (Between):              0.0364
No. Observations:              123457   R-squared (Within):               0.0163
Date:                Thu, May 05 2022   R-squared (Overall):              0.0218
Time:                        14:18:19   Log-likelihood                -7.207e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      231.52
Entities:                         300   P-value                           0.0000
Avg Obs:                       411.52   Distribution:                F(7,123143)
Min Obs:                       1.0000                                           
Max Obs:                       9280.0   F-statistic (robust):             163.48
                            